In [1]:
import numpy as np
import pandas as pd
import warnings
import time
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import math
import pickle
import statistics
import random
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sktime as sktime
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sktime.datasets import load_from_tsfile_to_dataframe
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.dists_kernels.scipy_dist import ScipyDist
from sktime.dists_kernels.compose_from_align import DistFromAligner
import os
import sys
%matplotlib inline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


### Load normal and abnormal datasets

In [2]:
final_normal_df = pd.read_pickle('normal_gait.pkl')
final_normal_df = final_normal_df.drop(columns='time')
final_abnormal_df = pd.read_pickle('abnormal_gait.pkl')
final_abnormal_df = final_abnormal_df.drop(columns='time')

,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
0,199.100,87.340000,91.806000,24.090,20.702000,87.670000,88.352000,63.778,163.900000,78.243,112.420000,49.995,13.750000,103.565000,144.408000,79.530000,662.838000,745.811000,1
1,196.889,86.020000,91.575000,23.914,20.911000,87.450000,89.276000,64.372,163.273000,77.550,112.420000,49.445,13.750000,104.390000,143.836000,79.530000,660.407000,744.194000,1
2,191.488,83.600000,89.232000,24.090,20.911000,87.901000,90.860000,64.570,162.019000,77.781,112.420000,50.820,13.750000,105.490000,146.410000,79.530000,652.652000,748.220000,1
3,201.476,88.220000,94.182000,24.618,20.493000,88.594000,85.646000,63.976,162.228000,77.550,112.145000,50.820,13.750000,103.312000,140.327000,77.341000,667.205000,737.473000,1
4,262.999,119.361000,115.676000,28.391,19.030000,92.884000,70.444000,59.917,157.157000,75.163,103.895000,50.545,10.175000,86.218000,102.905000,56.991000,768.702000,643.049000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77913,8.360,51.799000,37.961000,114.620,86.559000,69.564000,48.906000,30.514,5.170000,0.000,20.647000,77.242,81.191000,16.236000,21.648000,21.494000,448.283000,243.628000,72
77914,8.118,44.704000,35.398000,114.466,88.792000,72.105000,50.622000,30.910,4.906000,0.000,15.202000,81.455,87.164000,17.160000,23.320000,22.550000,445.115000,251.757000,72
77915,8.118,46.134000,37.092000,112.563,87.076000,69.542000,48.191000,30.778,4.906000,0.000,15.917000,84.062,87.670000,17.589000,23.716000,22.550000,439.494000,256.410000,72
77916,7.634,43.252000,35.277000,112.420,87.604000,69.696000,49.192000,30.778,4.510000,0.000,14.476000,86.977,91.553000,18.590000,25.432000,23.738000,435.853000,265.276000,72


,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
0,62.458,165.143,129.239,88.231,32.659,71.005,95.238,39.600,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,683.573,0.000,1
1,28.083,110.077,82.115,106.755,39.831,107.096,148.456,57.728,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,680.141,0.000,1
2,4.807,20.603,13.079,84.799,21.252,150.832,201.322,86.779,85.855,28.809,42.801,0.000,0.462,0.000,0.000,0.000,583.473,157.927,1
3,0.000,3.047,0.704,4.400,3.410,58.333,79.123,153.065,112.211,125.180,148.588,9.097,13.365,0.000,4.961,0.000,302.082,413.402,1
4,0.000,0.000,0.000,0.000,0.308,0.000,2.233,29.579,132.033,154.528,174.559,27.852,35.519,4.301,18.843,0.000,32.120,547.635,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107048,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,211.211,159.280,196.845,172.150,50.567,0.000,790.053,93
107049,108.042,84.755,63.833,11.330,8.613,0.253,0.286,0.550,0.000,0.000,0.000,78.485,78.584,249.689,179.146,66.902,277.662,652.806,93
107050,106.579,149.633,131.538,72.083,63.910,3.850,14.575,5.489,0.000,0.000,0.000,13.156,17.182,31.229,14.025,57.431,547.657,133.023,93
107051,66.616,127.677,99.264,118.360,91.025,26.323,52.580,11.814,0.000,0.000,0.000,0.000,0.990,0.462,0.000,1.144,593.659,2.596,93


In [3]:
abnormal_indices = (final_abnormal_df['Subject']).unique()
abnormal_no = (final_abnormal_df['Subject']).nunique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93])

93

In [4]:
# Specify the file path of the saved pkl file
file_path = 'final_bins_1.pkl'

# Load the dictionary from the pkl file
with open(file_path, 'rb') as file:
    final_numbers_dictionary = pickle.load(file)
# Now, 'loaded_dictionary' contains the dictionary loaded from the pkl file
display("Dictionary loaded from pickle file:")

'Dictionary loaded from pickle file:'

{'sample_numbers_list_0.0_to_0.1': [42, 45, 11, 21, 47, 49],
 'sample_numbers_list_0.1_to_0.2': [12, 70, 13, 38, 20, 31, 44, 57, 58],
 'sample_numbers_list_0.2_to_0.3': [36, 2, 6, 62, 26],
 'sample_numbers_list_0.3_to_0.4': [5, 17, 37, 56],
 'sample_numbers_list_0.4_to_0.5': [18, 3, 8, 43, 52],
 'sample_numbers_list_0.5_to_0.6': [40, 71],
 'sample_numbers_list_0.6_to_0.7': [14, 72, 46, 27, 1, 9, 23],
 'sample_numbers_list_0.7_to_0.8': [15, 4, 32, 33, 59, 25, 54],
 'sample_numbers_list_0.8_to_0.9': [67, 7, 16, 48, 53, 63],
 'sample_numbers_list_0.9_to_1.0': [19, 64],
 'sample_numbers_list_1.0_to_1.1': [10, 30, 34, 61],
 'sample_numbers_list_1.1_to_1.2': [28, 60, 41, 24],
 'sample_numbers_list_1.2_to_1.3': [51],
 'sample_numbers_list_1.3_to_1.4': [29, 50],
 'sample_numbers_list_1.4_to_1.5': [68, 22],
 'sample_numbers_list_1.5_to_1.6': [55, 39],
 'sample_numbers_list_1.6_to_1.7': [],
 'sample_numbers_list_1.7_to_1.8': [65],
 'sample_numbers_list_1.8_to_1.9': [],
 'sample_numbers_list_1.9_

### Function to find the length of the time series

In [5]:
# Define a function to z-normalize a column
def z_normalize(column):
    mean = column.mean()
    std = column.std()
    z_normalized = (column - mean) / std
    return z_normalized

In [6]:
def time_series_length_finding(df):
    train_1_abnormal = df.copy()
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    row_count = []
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        no_rows = train_1_abnormal_new.shape[0]
        row_count.append(no_rows)
    display("Avg length:",statistics.mean(row_count))
    avg_len_ts = int(statistics.mean(row_count))
    return(avg_len_ts)

In [7]:
def preprocessing_normal_df(df,desired_length):
    train_2_normal = df.copy()
    df_train_3 = pd.DataFrame(columns = train_2_normal.columns)
    num_dim = df_train_3.shape[1]
    display(num_dim)
    df_train_3.columns = range(len(df_train_3.columns))
    train_2_sub = (train_2_normal['Subject'].unique()).tolist()
    m = 0
    for i in train_2_sub:
        train_2_normal_new = train_2_normal[(train_2_normal['Subject']== i)]
        df_row_count = train_2_normal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            train_2_normal_new.tail()
            last_row = train_2_normal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_2_normal_initial = pd.concat([train_2_normal_new, new_rows_df], ignore_index=True)
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
        else:
            train_2_normal_initial = train_2_normal_new.iloc[:desired_length,:]
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
            
        train_2_normal_final.columns = range(len(train_2_normal_final.columns))
        
        df_train_3.at[m,0] = i
        j = 0
        while j < num_dim:
            df_train_3.at[m,j] = train_2_normal_final[j]
            j += 1
        m += 1    
    df_train_3['class'] = 'normal'
    df_normal_baseline = df_train_3.iloc[:,1:]
    return(df_normal_baseline)

In [8]:
def preprocessing_dataframe(df,desired_length):
    train_1_abnormal = df.copy()
    df_train_2 = pd.DataFrame(columns = train_1_abnormal.columns)
    num_dim_ab = df_train_2.shape[1]

    df_train_2.columns = range(len(df_train_2.columns))
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    m = 0
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        df_row_count = train_1_abnormal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            last_row = train_1_abnormal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_1_abnormal_initial = pd.concat([train_1_abnormal_new, new_rows_df], ignore_index=True)
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        else:
            train_1_abnormal_initial = train_1_abnormal_new.iloc[:desired_length,:]
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        train_1_abnormal_final.columns = range(len(train_1_abnormal_final.columns))
        
        df_train_2.at[m,0] = i
        j = 0
        while j < num_dim_ab:
            df_train_2.at[m,j] = train_1_abnormal_final[j]
            j += 1
        m += 1    
    df_train_2['class'] = 'abnormal'
    df_final = df_train_2.iloc[:,1:]
    return(df_final)

In [9]:
def ts_classifier(df_class_1):
    accuacy_list_1 = []
    precision_list_1 = []
    recall_list_1 = []
    f1_list_1 = []

    X_1 = df_class_1.iloc[:,:-1]
    y_1 = df_class_1.iloc[:,-1]
    
    display("X_1:",X_1)

    for i in range(1,31):
        X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=i,stratify = y_1)

        dtw_knn = KNeighborsTimeSeriesClassifier(distance="dtw")
        dtw_knn.fit(X_train_1, y_train_1)
        y_pred_1 = dtw_knn.predict(X_test_1)

        pos_label = 'abnormal'

        acc_1 = accuracy_score(y_test_1, y_pred_1)
        accuacy_list_1.append(acc_1)

        prec_1 = precision_score(y_test_1, y_pred_1, pos_label=pos_label)
        precision_list_1.append(prec_1)

        recall_1 = recall_score(y_test_1, y_pred_1, pos_label=pos_label)
        recall_list_1.append(recall_1)

        f1_1 = f1_score(y_test_1, y_pred_1, pos_label=pos_label)
        f1_list_1.append(f1_1)
        display('f1_score: %f' % f1_1)
        display("Completed",i)

    acc_mean_1 = np.mean(accuacy_list_1)
    acc_std_dev_1 = np.std(accuacy_list_1)

    prec_mean_1 = np.mean(precision_list_1)
    prec_std_dev_1 = np.std(precision_list_1)

    recall_mean_1 = np.mean(recall_list_1)
    recall_std_dev_1 = np.std(recall_list_1)

    f1_mean_1 = np.mean(f1_list_1)
    f1_std_dev_1 = np.std(f1_list_1)
    display('F1 Mean: %f' % f1_mean_1)
    return(acc_mean_1,acc_std_dev_1,prec_mean_1,prec_std_dev_1,recall_mean_1,recall_std_dev_1,f1_mean_1,f1_std_dev_1)

In [10]:
# Keys to combine
key_to_combine1 = 'sample_numbers_list_0.0_to_0.1'
key_to_combine2 = 'sample_numbers_list_0.1_to_0.2'
key_to_combine3 = 'sample_numbers_list_0.2_to_0.3'
# Create a new dictionary with combined values
hb_dict = {
    'hb_key': final_numbers_dictionary[key_to_combine1] + final_numbers_dictionary[key_to_combine2]
+ final_numbers_dictionary[key_to_combine3]}

{'hb_key': [42,
  45,
  11,
  21,
  47,
  49,
  12,
  70,
  13,
  38,
  20,
  31,
  44,
  57,
  58,
  36,
  2,
  6,
  62,
  26]}

In [11]:
classifier_dict_hb = {}
key = 'hb_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(hb_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

20

,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
1212,287.980,204.600,100.540,49.940,8.470,72.050,16.720,22.880,33.330,30.47,40.700,56.540,13.750,110.990,65.450,55.110,763.180,406.340,2
1213,309.507,243.133,101.508,62.260,9.724,76.065,15.488,20.944,15.202,11.44,17.193,23.166,4.675,69.245,40.458,35.354,838.629,216.733,2
1214,325.105,278.278,100.342,77.110,10.142,77.880,14.036,18.304,6.578,0.00,5.214,0.000,0.000,1.650,2.541,2.101,901.197,18.084,2
1215,295.449,266.970,99.385,88.836,11.187,96.239,18.414,22.000,4.994,0.00,2.838,0.000,0.000,0.000,1.012,0.000,898.480,8.844,2
1216,259.138,250.690,91.586,108.273,14.531,115.115,25.806,24.464,2.387,0.00,0.000,0.000,0.000,0.000,0.000,0.000,889.603,2.387,2


,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
3638,379.060,152.020,194.076667,46.676667,9.313333,33.183333,46.603333,11.990,0.0,4.180,0.0,12.540,18.333333,49.536667,81.216667,93.500,872.923333,259.306667,4
3639,390.291,184.426,213.455000,63.701000,21.010000,60.599000,76.362000,19.580,0.0,0.418,0.0,3.234,3.366000,13.794000,18.722000,13.651,1029.424000,53.185000,4
3640,292.897,159.698,188.870000,91.861000,50.094000,111.364000,151.019000,41.437,0.0,0.000,0.0,0.000,0.000000,3.223000,5.214000,0.000,1087.240000,8.437000,4
3641,136.301,85.756,90.002000,146.773000,85.877000,185.295000,242.814000,84.480,0.0,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000,1057.298000,0.000000,4
3642,23.540,15.576,12.540000,131.373000,95.546000,232.188000,326.557000,118.591,0.0,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000,955.911000,0.000000,4


'Avg length:'

1169.8

...

29

'f1_score: 0.800000'

'Completed'

30

'F1 Mean: 0.611693'

In [12]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'hb_gait_dtwknn_normalized.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_hb, file)

print(f"Dictionary saved to {pickle_file_path}")

In [13]:
import pickle

# Specify the file path
file_path = "hb_gait_dtwknn_normalized.pickle"

# Open the file in binary mode and load the data
with open(file_path, "rb") as file:
    data = pickle.load(file)

{'hb_key': {'acc_mean': 0.6625,
  'acc_std_dev': 0.15189771777965155,
  'prec_mean': 0.6866666666666668,
  'prec_std_dev': 0.21638443156156642,
  'recall_mean': 0.5916666666666667,
  'recall_std_dev': 0.2539958442354695,
  'f1_mean': 0.6116931216931217,
  'f1_std_dev': 0.21061603378334448}}

In [14]:
classifier_dict_hb

{'hb_key': {'acc_mean': 0.6625,
  'acc_std_dev': 0.15189771777965155,
  'prec_mean': 0.6866666666666668,
  'prec_std_dev': 0.21638443156156642,
  'recall_mean': 0.5916666666666667,
  'recall_std_dev': 0.2539958442354695,
  'f1_mean': 0.6116931216931217,
  'f1_std_dev': 0.21061603378334448}}

In [15]:
all_values = [value for sublist in final_numbers_dictionary.values() for value in sublist]
healthy_dict = {'healthy_key': all_values}

{'healthy_key': [42,
  45,
  11,
  21,
  47,
  49,
  12,
  70,
  13,
  38,
  20,
  31,
  44,
  57,
  58,
  36,
  2,
  6,
  62,
  26,
  5,
  17,
  37,
  56,
  18,
  3,
  8,
  43,
  52,
  40,
  71,
  14,
  72,
  46,
  27,
  1,
  9,
  23,
  15,
  4,
  32,
  33,
  59,
  25,
  54,
  67,
  7,
  16,
  48,
  53,
  63,
  19,
  64,
  10,
  30,
  34,
  61,
  28,
  60,
  41,
  24,
  51,
  29,
  50,
  68,
  22,
  55,
  39,
  65,
  66,
  69,
  35]}

In [16]:
classifier_dict_healthy = {}
key = 'healthy_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(healthy_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}
    display(f1_mean)

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)
    display(f1_mean)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

72

,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
0,199.100,87.340,91.806,24.090,20.702,87.670,88.352,63.778,163.900,78.243,112.420,49.995,13.750,103.565,144.408,79.530,662.838,745.811,1
1,196.889,86.020,91.575,23.914,20.911,87.450,89.276,64.372,163.273,77.550,112.420,49.445,13.750,104.390,143.836,79.530,660.407,744.194,1
2,191.488,83.600,89.232,24.090,20.911,87.901,90.860,64.570,162.019,77.781,112.420,50.820,13.750,105.490,146.410,79.530,652.652,748.220,1
3,201.476,88.220,94.182,24.618,20.493,88.594,85.646,63.976,162.228,77.550,112.145,50.820,13.750,103.312,140.327,77.341,667.205,737.473,1
4,262.999,119.361,115.676,28.391,19.030,92.884,70.444,59.917,157.157,75.163,103.895,50.545,10.175,86.218,102.905,56.991,768.702,643.049,1


,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
0,62.458,165.143,129.239,88.231,32.659,71.005,95.238,39.600,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,683.573,0.000,1
1,28.083,110.077,82.115,106.755,39.831,107.096,148.456,57.728,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,680.141,0.000,1
2,4.807,20.603,13.079,84.799,21.252,150.832,201.322,86.779,85.855,28.809,42.801,0.000,0.462,0.000,0.000,0.0,583.473,157.927,1
3,0.000,3.047,0.704,4.400,3.410,58.333,79.123,153.065,112.211,125.180,148.588,9.097,13.365,0.000,4.961,0.0,302.082,413.402,1
4,0.000,0.000,0.000,0.000,0.308,0.000,2.233,29.579,132.033,154.528,174.559,27.852,35.519,4.301,18.843,0.0,32.120,547.635,1


'Avg length:'

1153.375

...

'f1_score: 0.400000'

'Completed'

30

'F1 Mean: 0.621803'

0.6218033713558934

In [17]:
classifier_dict_healthy

{'healthy_key': {'acc_mean': 0.6344827586206896,
  'acc_std_dev': 0.06815617048447631,
  'prec_mean': 0.6421173379996911,
  'prec_std_dev': 0.08675646775133784,
  'recall_mean': 0.619047619047619,
  'recall_std_dev': 0.12211910719313243,
  'f1_mean': 0.6218033713558934,
  'f1_std_dev': 0.08197449120446465}}

In [18]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'healthy_gait_dtwknn_normalized.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_healthy, file)

print(f"Dictionary saved to {pickle_file_path}")